In [1]:
import requests, sys
from lxml import etree
import pandas as pd
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from biopandas.pdb import PandasPdb
import torch
import numpy as np
from periodictable import elements
from torch_geometric.data import Data
import os
from tqdm import tqdm
from IPython.utils import io


In [ ]:
def build_xml(offset=0,size=200):
    requestURL = f"https://www.ebi.ac.uk/proteins/api/proteins?offset={offset}&size={size}&reviewed=true&isoform=0&seqLength=1-120"

    r = requests.get(requestURL, headers={ "Accept" : "application/xml"})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    responseBody = r.text
    
    file_path = 'output.xml'
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(responseBody)
    
    # print(responseBody)

In [ ]:
def get_atomic_structure(pdb_id):
    # Define the path for the local file storage
    local_file_path = f"PyMol/pdb_files_for_pymol/{pdb_id}.cif"
    # error_log_path = f"error_pdb_ids.txt"

    # with open(error_log_path, "r") as error_log_file:
    #     faulty_ids = {line.strip() for line in error_log_file}

    # if pdb_id in faulty_ids:
    #     print(f"Skipping fetch for known faulty PDB ID {pdb_id}.")
    #     return None

    # Check if the file exists locally
    # if not os.path.exists(local_file_path):
    #     # If the file does not exist, fetch it and save locally
    #     try:
    #         ppdb = PandasPdb().fetch_pdb(pdb_id.lower(),source="pdb")
    #         ppdb.to_pdb(path=local_file_path)
    #     except AttributeError:
    #         with open(error_log_path, "a") as error_log_file:
    #             error_log_file.write(f"{pdb_id}\n")
    # else:
    #     # If the file exists, load it from the local file
    #     ppdb = PandasPdb().read_pdb(local_file_path)

    # # Extract atomic structure
    # atom = ppdb.df['ATOM']
    # structure = list(zip(atom['element_symbol'], atom['x_coord'], atom['y_coord'], atom['z_coord']))
        # Initialize a list to hold information about each atom
    atom_info = []

    # Open and read the file
    with open(local_file_path, 'r') as file:
        # Iterate through each line in the file
        for line in file:
            # Check if the line starts with 'ATOM'
            if line.startswith("ATOM"):
                columns = line.split()

                # Extract the desired information
                atom_number = int(columns[1])
                atom_type = columns[2]
                x = float(columns[11])
                y = float(columns[12])
                z = float(columns[13])

                # Append a dictionary with the extracted info to our list
                atom_info.append((atom_type,x,y,z))

    return atom_info


def get_prot_analysis(sequence):
    analysis = ProteinAnalysis(sequence)
    mw = analysis.molecular_weight()
    pI = analysis.isoelectric_point()
    instability = analysis.instability_index()
    aromaticity = analysis.aromaticity()
    gravy = analysis.gravy()
    return mw,pI, instability,aromaticity,gravy



In [ ]:
def read_xml(filepath = "output.xml"):
    with open(filepath, 'r', encoding='utf-8') as file:
        xml_data_str = file.read()
        
    xml_data_bytes = xml_data_str.encode('utf-8')
    root = etree.fromstring(xml_data_bytes)
    
    namespaces = {
        'uniprot': "http://uniprot.org/uniprot",
        'xsi': "http://www.w3.org/2001/XMLSchema-instance"
    }
    
    # for protein_name in root.findall('.//uniprot:protein/uniprot:recommendedName/uniprot:fullName', namespaces):
    #     print(protein_name.text)
    
    df = pd.DataFrame(columns=['pdb','accession','name','sequence','coords',
                               #'mw','pI','II','aromaticity','gravy'
                               ])
    
    for entry in root.findall('uniprot:entry', namespaces):
        sequence = entry.find('uniprot:sequence', namespaces).text
        # print(sequence)
        accession = entry.find('uniprot:accession', namespaces).text
        # print(f"Accession: {accession}")
        name = entry.find('uniprot:name', namespaces).text
        # print(f"Name: {name}")
        pdb_ids = entry.findall(".//uniprot:dbReference[@type='PDB']", namespaces)
        for pdb_id in pdb_ids:
            # print(f"PDB ID: {pdb_id.get('id')}")
            df.loc[len(df.index)] = [pdb_id.get('id'),accession,name,sequence,None]
    df = df.drop_duplicates()
    
    # error_log_path = f"error_pdb_ids.txt"
    # with open(error_log_path, "r") as error_log_file:
    #     faulty_ids = {line.strip() for line in error_log_file}
    
    # for _,row in df.iterrows():


    return df


In [ ]:
df = pd.DataFrame()
for i in tqdm(range(500)):
    build_xml(0+i*200,200)
    df = pd.concat([df,read_xml()])
df

In [ ]:
df = df.drop_duplicates("pdb")
df

In [ ]:
df = df[:2000]

In [ ]:
current_pymol_results = pd.read_table("PyMol/Results_new.txt")

In [ ]:
current_pymol_results

In [ ]:
not_done_in_pymol = list(set(df.pdb)-set(current_pymol_results.PDB))
len(not_done_in_pymol)

In [ ]:
output = open("PyMol/pdb_ids.txt", "w")
for id in not_done_in_pymol:
    output.write(f"{id}\n")
output.close()

In [ ]:
os.system('pymol PyMol/PyMOL_Analysis.py')

In [2]:
res_df = pd.read_table("PyMol/Results_new.txt")

In [3]:
res_df

,PDB,No. a.a.,Glycine,S.S.,Long SS,Charge,SASA,No. pos.,No. Surf. pos.,Pos. area,No. neg.,No. Surf. neg.,Neg. area,No. hyd.,No. Surf. hyd.,hyd. area,Alpha,Beta,Salt bridges,H-bonds
0,7S4N,85,8,4.0,4,1,5121.14990234375,14,14,1203.4117431640625,13,11,1010.9176635742188,24,21,1318.6737060546875,7.317073170731708,51.21951219512195,3,67
1,7S58,85,8,4.0,4,1,5371.29345703125,14,14,1286.778076171875,13,12,1050.521240234375,24,22,1457.382080078125,9.523809523809524,47.61904761904762,7,74
2,7S59,85,8,4.0,4,1,5425.18896484375,14,14,1391.2445068359375,13,12,1189.390380859375,24,21,1214.5059814453125,9.63855421686747,48.19277108433735,1,2
3,7SO0,85,8,4.0,4,1,5613.6025390625,14,14,1432.86328125,13,12,1140.178955078125,24,22,1375.7874755859375,8.333333333333334,48.80952380952381,6,72
4,5N4B,731,57,0.0,N,-13,30260.47265625,79,77,6641.99560546875,92,87,6856.421875,296,206,6466.42138671875,26.59279778393352,32.27146814404432,85,810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998,1L3O,68,6,0.0,N,2,5811.89453125,10,10,1617.381103515625,8,8,804.2407836914062,19,19,1312.1702880859375,42.64705882352941,0.0,4,60
1999,2Z0G,112,12,0.0,N,-2,6510.53662109375,18,16,1516.2017822265625,20,18,1778.913818359375,46,34,2046.843505859375,40.0,37.89473684210526,16,98
2000,5HS9,110,5,0.0,N,-5,6642.74365234375,13,11,1380.4544677734375,18,12,1296.365234375,44,33,2108.976806640625,78.4090909090909,9.090909090909092,5,105
2001,7V33,433,51,0.0,N,0,18235.90234375,53,50,4857.62255859375,53,47,2817.515869140625,167,98,4164.13623046875,55.65819861431871,11.547344110854503,79,505


In [22]:
import re

In [23]:
def get_atomic_structure(pdb_id):
    try:
        if pdb_id == 'pdb' or pdb_id == "6zke":
            return None
        # Define the path for the local file storage
        local_file_path = f"PyMol/pdb_files_for_pymol/{pdb_id}.cif"
        atom_info = []

        # Open and read the file
        with open(local_file_path, 'r') as file:
            # Iterate through each line in the file
            for line in file:
                # Check if the line starts with 'ATOM'
                if line.startswith("ATOM"):
                    columns = line.split()

                    # Extract the desired information
                    atom_number = int(columns[1])
                    atom_type = columns[2]
                    x = float(columns[11])
                    y = float(columns[12])
                    z = float(columns[13])

                    # Append a dictionary with the extracted info to our list
                    if not re.search("[a-zA-Z]",atom_type):
                        print(pdb_id)
                        return None
                    atom_info.append((atom_type,x,y,z))
    except BaseException as err:
        print(pdb_id)
        print(err)
        return None

    if len(atom_info) >1500:
        return None
    return atom_info

In [24]:
res_df["coords"] = None

In [25]:
res_df

,PDB,No. a.a.,Glycine,S.S.,Long SS,Charge,SASA,No. pos.,No. Surf. pos.,Pos. area,...,No. Surf. neg.,Neg. area,No. hyd.,No. Surf. hyd.,hyd. area,Alpha,Beta,Salt bridges,H-bonds,coords
0,7S4N,85,8,4.0,4,1,5121.14990234375,14,14,1203.4117431640625,...,11,1010.9176635742188,24,21,1318.6737060546875,7.317073170731708,51.21951219512195,3,67,None
1,7S58,85,8,4.0,4,1,5371.29345703125,14,14,1286.778076171875,...,12,1050.521240234375,24,22,1457.382080078125,9.523809523809524,47.61904761904762,7,74,None
2,7S59,85,8,4.0,4,1,5425.18896484375,14,14,1391.2445068359375,...,12,1189.390380859375,24,21,1214.5059814453125,9.63855421686747,48.19277108433735,1,2,None
3,7SO0,85,8,4.0,4,1,5613.6025390625,14,14,1432.86328125,...,12,1140.178955078125,24,22,1375.7874755859375,8.333333333333334,48.80952380952381,6,72,None
4,5N4B,731,57,0.0,N,-13,30260.47265625,79,77,6641.99560546875,...,87,6856.421875,296,206,6466.42138671875,26.59279778393352,32.27146814404432,85,810,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998,1L3O,68,6,0.0,N,2,5811.89453125,10,10,1617.381103515625,...,8,804.2407836914062,19,19,1312.1702880859375,42.64705882352941,0.0,4,60,None
1999,2Z0G,112,12,0.0,N,-2,6510.53662109375,18,16,1516.2017822265625,...,18,1778.913818359375,46,34,2046.843505859375,40.0,37.89473684210526,16,98,None
2000,5HS9,110,5,0.0,N,-5,6642.74365234375,13,11,1380.4544677734375,...,12,1296.365234375,44,33,2108.976806640625,78.4090909090909,9.090909090909092,5,105,None
2001,7V33,433,51,0.0,N,0,18235.90234375,53,50,4857.62255859375,...,47,2817.515869140625,167,98,4164.13623046875,55.65819861431871,11.547344110854503,79,505,None


In [26]:
# res_df['coords'] = res_df.apply(lambda row: get_atomic_structure(str(row.PDB).lower()),axis=1)

# Initialize an empty list to store the coordinates
coords_list = []

# Iterate over each row in the DataFrame
for index, row in tqdm(res_df.iterrows(),total=len(res_df)):
    # Apply the get_atomic_structure function to the lowercase PDB value of the current row
    # print(row['PDB'])
    coords = get_atomic_structure(str(row['PDB']).lower())
    # Append the result to the coords_list
    coords_list.append(coords)

# Assign the list of coordinates to the 'coords' column of the DataFrame
res_df['coords'] = coords_list

 63%|██████▎   | 1256/2003 [00:58<00:20, 36.48it/s]

1qck
list index out of range


 95%|█████████▍| 1900/2003 [01:18<00:02, 35.24it/s]

2odg
invalid literal for int() with base 10: 'AND'


100%|██████████| 2003/2003 [01:21<00:00, 24.63it/s]


In [21]:
res_df[res_df.PDB=="2odg"]

,PDB,No. a.a.,Glycine,S.S.,Long SS,Charge,SASA,No. pos.,No. Surf. pos.,Pos. area,...,No. Surf. neg.,Neg. area,No. hyd.,No. Surf. hyd.,hyd. area,Alpha,Beta,Salt bridges,H-bonds,coords


In [12]:
output_df = res_df[~pd.isna(res_df["coords"])]

In [13]:
output_df.to_csv("scraped_proteins_dataset.csv")

In [14]:
output_df

,PDB,No. a.a.,Glycine,S.S.,Long SS,Charge,SASA,No. pos.,No. Surf. pos.,Pos. area,...,No. Surf. neg.,Neg. area,No. hyd.,No. Surf. hyd.,hyd. area,Alpha,Beta,Salt bridges,H-bonds,coords
3,7SO0,85,8,4.0,4,1,5613.6025390625,14,14,1432.86328125,...,12,1140.178955078125,24,22,1375.7874755859375,8.333333333333334,48.80952380952381,6,72,"[(N, -34.8217, -20.87944, 1.0), (C, -33.59635,..."
95,4X9Z,50,4,4.0,3,5,4485.33984375,7,7,883.1650390625,...,1,93.76268005371094,14,14,1754.4361572265625,0.0,42.85714285714285,3,25,"[(N, -6.438, 18.425, 1.0), (C, -5.456, 19.282,..."
145,5JYQ,18,1,1.0,0,2,1899.593505859375,3,3,537.2381591796875,...,1,51.80294799804688,5,5,477.1363220214844,77.77777777777777,0.0,2,16,"[(N, 38.785, 34.309, 1.0), (C, 39.14, 33.733, ..."
157,6XPK,123,9,0.0,N,-12,5449.1162109375,6,4,486.8616638183594,...,14,1327.5223388671875,54,30,1684.3687744140625,42.0,33.0,2,111,"[(N, 4.075, 16.801, 1.0), (C, 4.516, 15.921, 1..."
158,6XPL,123,9,0.0,N,-11,5360.82275390625,6,4,524.8343505859375,...,13,1197.3155517578125,55,31,1802.7811279296875,42.85714285714285,33.673469387755105,2,107,"[(N, 0.851, 9.7, 1.0), (C, 0.625, 8.758, 1.0),..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,1RDS,105,12,2.0,1,-17,5547.0419921875,3,3,99.17447662353516,...,20,1651.90234375,37,29,1252.311279296875,16.19047619047619,32.38095238095238,3,107,"[(N, 60.723, 20.075, 1.0), (C, 61.018, 20.866,..."
1992,1CIH,107,12,0.0,N,6,6534.4716796875,17,17,2100.885986328125,...,11,1084.4703369140625,36,33,1232.388916015625,47.66355140186916,6.542056074766355,2,111,"[(N, 12.163, -8.75, 1.0), (C, 11.692, -7.4, 1...."
1996,1D3W,106,3,0.0,N,-18,5577.23681640625,7,6,605.6025390625,...,25,2279.356201171875,39,30,1023.7701416015625,39.62264150943396,13.20754716981132,5,95,"[(N, 19.045, 5.79, 1.0), (C, 17.717, 5.635, 1...."
1997,4F6B,110,12,0.0,N,1,6181.40087890625,11,11,1366.2310791015625,...,10,725.13671875,51,44,1796.08203125,81.81818181818181,0.0,6,147,"[(N, 56.036, 14.521, 1.0), (C, 54.717, 15.152,..."


In [ ]:
def build_dataset(df):
    df = df.copy().reset_index()
    element_translation = {el.symbol.lower(): el.number for el in elements}
    
    data_list=list()
    for j, row in df.iterrows():
        name = row['pdb']
        y = torch.Tensor([row['gravy'], row['mw'], row['pI'], row['II'], row['aromaticity'],row['Alpha']])
        
        n_atoms = len(row['coords'])
        z = torch.empty((n_atoms)); pos = torch.empty((n_atoms, 3)) 
        for i, x in enumerate(row['coords']):
            z[i] = torch.Tensor([element_translation[x[0].lower()]])
            pos[i] = torch.Tensor([x[1], x[2], x[3]])
        
        data=Data(
            z=z,
            pos=pos,
            y=y,
            name=name,
            idx=j
        )
        data_list.append(data)
    
    return data_list
